In [32]:
# import library
import pandas as pd
import numpy as np 
import matlab 
import matplotlib as plt
import os
import numpy as np


'2.0.0'

In [ ]:
# read the file
# mortality
mortality_path = r'data/Morticd10_part1.csv'
mortality_data_2002 = pd.read_csv(mortality_path, low_memory=False)
mortality_data_2002.head()

In [ ]:
# population
pop_path = r'data/pop.csv'
pop_data = pd.read_csv(pop_path, low_memory=False)

In [ ]:
# country code
country_code_path = r'data/country_codes.csv'
country_code = pd.read_csv(country_code_path, low_memory=False)

In [ ]:
# changes to the mortality data file, creating a new data frame with only the number of deaths and the population, country, year, deaths1 - deaths26, pop1 - pop 26, im_death
# creating the country code map 
country_code_map = {}
for index, rows in country_code.iterrows():
    country_code_map[rows['country']] = rows['name']
    
len(country_code_map)

In [ ]:
# looping through the code map to locate each different countries
# condense the mortality and pop rate 
undesired = ['Admin1', 'SubDiv', 'Sex', 'List', 'Frmat', 'IM_Frmat', 'Cause']
modified_dict = {}
for i in mortality_data_2002.columns.values:
    if i not in undesired:
       modified_dict[i] = []

modified_df = pd.DataFrame(modified_dict)
list_delete_keys = []

for key in country_code_map: 
    temp_mortality_df = mortality_data_2002[mortality_data_2002['Country'] == key] # filter to the country
    if temp_mortality_df.empty:
        list_delete_keys.append(key)
    else:
        for i in range (1980, 2003):
            print(key)
            print(i)
            temp_temp_mortality_df = temp_mortality_df.loc[temp_mortality_df['Year'] == i] # filter to the year 
            if temp_temp_mortality_df.empty:
                continue
            else:
                modified_df.loc[modified_df.shape[0]] = ['None'] * modified_df.shape[1]
                modified_df.loc[modified_df.shape[0]-1, ['Country']] = key
                modified_df.loc[modified_df.shape[0]-1, ['Year']] = i
                column_name = modified_df.columns.values 
                for c in column_name[2:]:
                    if c in temp_temp_mortality_df.columns.values:
                        modified_df.loc[modified_df.shape[0]-1, [c]] = temp_temp_mortality_df[c].sum()
            
                # modified_df.index = modified_df.index + 1
                # modified_df = modified_df.sort_index()
                    
modified_df           

In [ ]:
# delete keys from dictionary
for key in list_delete_keys:
    del country_code_map[key]

In [54]:
# building the mortality model 
country_mortality_model = {}
for key in country_code_map:
    # isolate country
    temp_modified_df = modified_df[modified_df['Country'] == key]
    temp_modified_df.drop(columns=['Country'], inplace = True) 
    
    # finding alpha 
    last_year = 2002
    first_year = temp_modified_df['Year'].min()
    duration = last_year - first_year
    last_row = temp_modified_df[temp_modified_df['Year'] == last_year]
    first_row = temp_modified_df[temp_modified_df['Year'] == first_year]
    aphla = last_row.sub(first_row)
    display(aphla)
    
    # finding beta and k 
    m_hat = (last_row - aphla)
    display(m_hat)
    s, u, v = np.linalg.svd(m_hat)
    scalar = s[1][1]
    k = scalar * u[1]
    b = v[1] / scalar 
    
    country = country_code_map[key]
    country_mortality_model[country] = [aphla, b, k]
    
country_mortality_model
    
    
        

C:\Users\Public\Documents\Wondershare\CreatorTemp\ipykernel_26400\2951213682.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_modified_df.drop(columns=['Country'], inplace = True)


,Year,Deaths1,Deaths2,Deaths3,Deaths4,Deaths5,Deaths6,Deaths7,Deaths8,Deaths9,...,Deaths21,Deaths22,Deaths23,Deaths24,Deaths25,Deaths26,IM_Deaths1,IM_Deaths2,IM_Deaths3,IM_Deaths4
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


,Year,Deaths1,Deaths2,Deaths3,Deaths4,Deaths5,Deaths6,Deaths7,Deaths8,Deaths9,...,Deaths21,Deaths22,Deaths23,Deaths24,Deaths25,Deaths26,IM_Deaths1,IM_Deaths2,IM_Deaths3,IM_Deaths4
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


UFuncTypeError: Cannot cast ufunc 'svd_m_f' input from dtype('O') to dtype('float64') with casting rule 'same_kind'